In [1]:
import os
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import pickle
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
def fetch_data(stock_tkr):
    url = 'https://finance.yahoo.com/quote/'+stock_tkr+'/history?period1=1390636800&period2=1548403200&interval=1d&filter=history&frequency=1d'
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    first_elem = driver.find_element_by_xpath('//*[@id="atomic"]')
    data_table = None
    rows = None
    prev_rows = 0
    while True:
        first_elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        data_table = driver.find_element_by_xpath('//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[2]')
        data = data_table.find_element_by_tag_name('tbody')
        rows = data.find_elements_by_tag_name('tr')
        curr_rows = len(rows)
        if curr_rows == prev_rows and count == 10:
            break
        elif curr_rows != prev_rows:
            count = 0
        prev_rows = curr_rows
        count += 1
    return data_table

def get_headers(data):
    head = data.find_element_by_tag_name('thead')
    head_line = head.find_element_by_tag_name('tr')
    headers = head_line.find_elements_by_tag_name('th')
    headers = [h.text for h in headers]
    return headers
    
def build_dataframe(data, headers):
    table = data.find_element_by_tag_name('tbody')
    rows = table.find_elements_by_tag_name('tr')
    res = []
    for row in rows:
        res.append(row.text.replace(', ', ':').replace(' ',':',1).split())
    return pd.DataFrame(res, columns=headers)

cwd = os.getcwd()
target_stocks = {'netflix':'NFLX', 'google':'GOOGL', 'apple':'AAPL', 'amazon':'AMZN', 'facebook':'FB', 'sp500':'^GSPC'}
for stock in target_stocks:
    data = fetch_data(target_stocks[stock])
    headers = get_headers(data)
    df = build_dataframe(data, headers)
    path_pickle = cwd + '/../data/{}.pk'.format(stock)
    df.to_pickle(path_pickle)